In [1]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
from pyAgrum.lib.pretty_print import pretty_cpt

from IPython.display import display, Math, Latex

import pandas
import numpy as np

# Exercice 7
> From Roland Donnat

L'objectif de cet exercice est d'utiliser un RB afin de prédire le résultat de matchs de football de Ligue 1 (division 1 de football en France) à partir de certaines statistiques de jeu.

Deux jeux de données vont être utilisés :

+ Un jeu de données d'apprentissage permettant de calibrer le modèle prédictif contanant l'historique des matchs des saisons 2005-2006 à 2009-2010 - Fichier fra_l1_app.csv
+ Un jeu de données de test permettant d'évaluer le modèle contanant les matchs des saisons 2010-2011 et 2011-201 - Fichier fra_l1_test.csv



### Prétraitement des données

Pour un apprentissage "facile", nous allons :
+ transformer toutes les données en entier ou chaîne, 
+ s'assurer que la base de tests ne possèdent pas de valeurs inconnues pour la base d'apprentissage
+ sélectionner les variables "HomeTeam","AwayTeam","HST","AST" qui nous serviront à tenter de prédire "FTR" 
+ créer les fichiers "learn.csv" et "test.csv" épurés.

In [2]:
# data file is in the ../resources folder

tests=pandas.read_csv("fra_l1_test.csv")

In [3]:
datas.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HY,AY,HR,AR
0,2005-07-29,Paris SG,Metz,4,1,H,2,0,H,16,11,9,7,2,3,0,0
1,2005-07-30,Marseille,Bordeaux,0,2,A,0,1,A,7,9,3,4,2,4,1,0
2,2005-07-30,Nancy,Monaco,0,1,A,0,0,D,10,5,6,3,2,2,2,0
3,2005-07-30,Nantes,Lens,2,0,H,2,0,H,10,10,3,3,1,3,0,0
4,2005-07-30,Nice,Troyes,1,1,D,1,0,H,13,5,6,5,2,2,0,1


In [4]:
tests.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HY,AY,HR,AR
0,2010-08-07,Lyon,Monaco,0,0,D,0,0,D,16,9,5,4,2,3,0,0
1,2010-08-07,Marseille,Caen,1,2,A,0,0,D,16,12,3,2,2,3,0,0
2,2010-08-07,Nice,Valenciennes,0,0,D,0,0,D,10,11,2,1,0,3,0,0
3,2010-08-07,Paris SG,St Etienne,3,1,H,2,1,H,19,14,6,5,2,1,0,0
4,2010-08-07,Rennes,Lille,1,1,D,1,0,H,20,14,5,5,3,3,0,0


### Prétraitement

L'apprentissage avec pyAgrum fonctionne sur des csv totalement prétraités (des entiers). Il s'agit donc de préparer la base en ce sens.

In [5]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1899 entries, 0 to 1898
Data columns (total 17 columns):
Date        1899 non-null object
HomeTeam    1899 non-null object
AwayTeam    1899 non-null object
FTHG        1899 non-null int64
FTAG        1899 non-null int64
FTR         1899 non-null object
HTHG        1899 non-null int64
HTAG        1899 non-null int64
HTR         1899 non-null object
HS          1899 non-null float64
AS          1899 non-null float64
HST         1899 non-null float64
AST         1899 non-null float64
HY          1899 non-null int64
AY          1899 non-null int64
HR          1899 non-null int64
AR          1899 non-null int64
dtypes: float64(4), int64(8), object(5)
memory usage: 267.0+ KB


In [6]:
#pretraitement

# float en int
datas["HS"] = datas["HS"].astype(int)
datas["AS"] = datas["AS"].astype(int)
tests["HS"] = tests["HS"].astype(int)
tests["AS"] = tests["AS"].astype(int)
datas["HST"] = datas["HST"].astype(int)
datas["AST"] = datas["AST"].astype(int)
tests["HST"] = tests["HST"].astype(int)
tests["AST"] = tests["AST"].astype(int)
    
# s'assurer que les valeurs de tests sont incluses dans les valeurs d'apprentissage
HST_app_max = np.max(datas["HST"])
AST_app_max = np.max(datas["AST"])
tests.loc[tests["HST"] > HST_app_max, "HST"] = HST_app_max
tests.loc[tests["AST"] > AST_app_max, "AST"] = AST_app_max

# ville -> entiers
towns = datas['HomeTeam'].unique()
towns2nums = dict(zip(towns,range(len(towns))))
nums2towns = dict(zip(range(len(towns)),towns))
datas['HomeTeamNum']=datas['HomeTeam'].replace(towns2nums)
datas['AwayTeamNum']=datas['AwayTeam'].replace(towns2nums)
tests['HomeTeamNum']=tests['HomeTeam'].replace(towns2nums)
tests['AwayTeamNum']=tests['AwayTeam'].replace(towns2nums)

# classe -> entiers
classes=datas['FTR'].unique()
classes2nums=dict(zip(classes,range(len(classes))))
datas['FTRNum']=datas['FTR'].replace(classes2nums)
tests['FTRNum']=tests['FTR'].replace(classes2nums)

datas.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HY,AY,HR,AR,HomeTeamNum,AwayTeamNum,FTRNum
0,2005-07-29,Paris SG,Metz,4,1,H,2,0,H,16,11,9,7,2,3,0,0,0,16,0
1,2005-07-30,Marseille,Bordeaux,0,2,A,0,1,A,7,9,3,4,2,4,1,0,1,14,1
2,2005-07-30,Nancy,Monaco,0,1,A,0,0,D,10,5,6,3,2,2,2,0,2,11,1
3,2005-07-30,Nantes,Lens,2,0,H,2,0,H,10,10,3,3,1,3,0,0,3,13,0
4,2005-07-30,Nice,Troyes,1,1,D,1,0,H,13,5,6,5,2,2,0,1,4,12,2


In [7]:
tests.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HY,AY,HR,AR,HomeTeamNum,AwayTeamNum,FTRNum
0,2010-08-07,Lyon,Monaco,0,0,D,0,0,D,16,9,5,4,2,3,0,0,17,11,2
1,2010-08-07,Marseille,Caen,1,2,A,0,0,D,16,12,3,2,2,3,0,0,1,23,1
2,2010-08-07,Nice,Valenciennes,0,0,D,0,0,D,10,11,2,1,0,3,0,0,4,21,2
3,2010-08-07,Paris SG,St Etienne,3,1,H,2,1,H,19,14,6,5,2,1,0,0,0,6,0
4,2010-08-07,Rennes,Lille,1,1,D,1,0,H,20,14,5,5,3,3,0,0,18,8,2


Dans un second temps, nous décidons de nous limiter aux variables sivantes :
+ features : "HomeTeamNum","AwayTeamNum","HST","AST"
+ classe : "FTRNum"

In [8]:
#variables selection
vars=["HomeTeamNum","AwayTeamNum","HST","AST","FTRNum"]

#sauvegarde des bases opérationnelles 
datas[vars].to_csv("learn.csv",index=False)
tests[vars].to_csv("test.csv",index=False)


## Modèle "Naive Bayes"
Nous essayons tout d'abord un modèle "Naïve Bayes" qui est connu pour ne pas être correct du tout théoriquement mais, finalement, pour assez bien fonctionner. Le principe du modèle est de supposer une structure du modèle où la variable à expliquer (la classe) est l'unique parents des variables explicantes (les features).

## Apprentissage 

Un apprentissage de structure peut-il donner mieux pour notre problème ?